1. Load up the 1K jeopardy dataset that has 1000 objects in total, keep at least the question, answer and round properties.
2. How do you check for the number of objects stored in the database?
3. Search for objects that are close to the concept of “spicy food recipes” and show 4 QnA
4. Can you find “spicy food recipes” related questions that were used in Double Jeopardy rounds?


### Q1: Load up the dataset, keep at least the question, answer and round properties.

In [ ]:
import requests
import json

# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/intro-workshop/main/data/jeopardy_1k.json')
data = json.loads(resp.text)  # Load data

# Parse the JSON and preview it
print(type(data), len(data))
print(json.dumps(data[1], indent=2))

### Connecting to Weaviate
We use the Weaviate Python client to connect to an **Embedded Weaviate** instance. This allows us to run a vector database locally without separate installation. We also provide an OpenAI API key in the headers so Weaviate can automatically generate vector embeddings for our text using the `text2vec-openai` module.

In [ ]:
import weaviate
from weaviate import EmbeddedOptions
import os

client = weaviate.Client(
    embedded_options=EmbeddedOptions(),
    additional_headers={
        "X-OpenAI-Api-Key": os.environ.get("OPENAI_API_KEY", "YOUR_OPENAI_API_KEY")
    }
)

In [ ]:
if client.schema.exists("Question"):
    client.schema.delete_class("Question")

### Defining the Schema
In Weaviate, we define a **Class** (similar to a table) and its **Properties** (columns). By setting the `vectorizer` to `text2vec-openai`, we tell Weaviate to convert our text properties into high-dimensional vectors automatically.



In [ ]:
# Define the class that will be used to add the data
class_definition = {
    "class": "Question",
    "vectorizer": "text2vec-openai",
    "properties": [
        {"name": "question", "dataType": ["text"]},
        {"name": "answer", "dataType": ["text"]},
        {"name": "round", "dataType": ["text"]}
    ]
}

client.schema.create_class(class_definition)

In [ ]:
# Insert the data into Weaviate using Batch processing for efficiency
with client.batch() as batch:
    for o in data:
        properties = {
            "question": o["Question"],
            "answer": o["Answer"],
            "round": o["Round"]
        }
        batch.add_data_object(properties, "Question")

### Q2. How do you check for the number of objects stored in the database?

### Data Aggregation
To count objects, Weaviate uses the `aggregate` function. This is more efficient than fetching all records. We use `with_meta_count()` to retrieve the total number of objects in the specified class.

In [ ]:
result = client.query.aggregate("Question").with_meta_count().do()
print(json.dumps(result, indent=2))

### 3. Search for objects that are close to the concept of "spicy food recipes" and show 4 QnA

### Semantic Search with `nearText`
Unlike keyword search, semantic search finds objects based on meaning. The `nearText` operator calculates the distance between the vector of our query and the vectors of the stored Jeopardy questions.



In [ ]:
response = (
    client.query
    .get("Question", ["question", "answer"])
    .with_near_text({"concepts": ["spicy food recipes"]})
    .with_limit(4)
    .do()
)

print(json.dumps(response, indent=2))

### 4. Can you find "spicy food recipes" related questions that were used in Double Jeopardy rounds?

### Filtered Vector Search
We can combine semantic search with standard filters (metadata filters). Here, we use a `where` filter to restrict results to the "Double Jeopardy!" round while still performing a conceptual search for "spicy food."

In [ ]:
where_filter = {
    "path": ["round"],
    "operator": "Equal",
    "valueString": "Double Jeopardy!"
}

response = (
    client.query
    .get("Question", ["question", "answer", "round"])
    .with_near_text({"concepts": ["spicy food recipes"]})
    .with_where(where_filter)
    .with_limit(4)
    .do()
)

print(json.dumps(response, indent=2))